<center> <h1> <b> CS114.L21: MACHINE LEARNING <center> <h1> <b>

<center> <h2> <b> BÀI TẬP COLAB SỐ 04: WEB SCRAPING (Ngày 07/6/2021) <center> <h> <b>

###**1. Nhóm thực hiện**

* 19522246 - Vũ Nguyễn Nhật Thanh
* 19522180 - Trương Thế Tấn
* 19521551 - Nông Thanh Hồng

#**I. Trang châm biếm**


###*Code dùng chung cho trang tin Châm biếm*

- **Thư viện chung:**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

- **Lưu data thành file .csv**

In [ ]:
df = pd.DataFrame({'is_sarcastic': 1,
                   'title': title, 
                   'article_link': article_link,
                   'publishdate': publishdate})

#Code được viết trên colab nên dẫn link lưu lại qua drive
df.to_csv("/content/gdrive/MyDrive/Colab Notebooks/Machine Learning/31062021_sarcasm-detection/dataset/<name>.csv", header=True, index=False)
df.to_csv("/content/<name>.csv", header=True, index=False)

##**1. The Beaverton**

- https://www.thebeaverton.com/



In [ ]:
#https://www.datacamp.com/community/tutorials/tutorial-python-beautifulsoup-datacamp-tutorials
title = []
article_link = []
publishdate = []

for cp in np.arange(1,500):
    url = "https://thebeaverton.com/page/" + str(cp)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')

    header = soup.findAll(class_ = "post-title entry-header")
    datetime = soup.findAll("aside", {"class": "post-author cf"})

    for i in header:
      article_link.append(i.a['href'])
      title.append(i.a['title'])

    for t in datetime:
      publishdate.append(t.time['datetime'])
    for t in range(111):
      publishdate.append("NaN")


##**2. The Burrard Street Journal**

- https://www.burrardstreetjournal.com/

In [ ]:
title = []
article_link = []
publishdate = []

for cp in np.arange(1,48):
    url = "https://www.burrardstreetjournal.com/page/" + str(cp)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')

    header = soup.findAll(class_ = "entry-title td-module-title")
    datetime = soup.findAll("span", {"class": "td-post-date"})

    for i in header:
      article_link.append(i.a['href'])
      title.append(i.a['title'])


    for t in datetime:
      publishdate.append(t.time['datetime'])

    if (cp == 1):
      for t in range(len(title) - len(publishdate)):
        publishdate.append("NaN")

##**3. The Dailyer:**

- http://thedailyer.com/


In [ ]:
#Link mẫu của The dailyer
##http://thedailyer.com/category/local/page/2/

title = []
article_link = []

# category: 6
category = ["world", "feat", "local", "nat", "sport", "opinion"]
for j in range(6):
  print (category[j])
  for cp in range(1,75):
    print (cp)
    url = "http://thedailyer.com/category/" + category[j] + "/page/" + str(cp)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')

    header = soup.findAll(class_ = "title entry-title")

    for i in header:
      article_link.append(i.a['href'])
      title.append(i.text)

    # do mỗi category có số trang khác nhau phải kiểm tra và thoát để không bị lỗi 404
    if j == 0 and cp == 6:
      break
    elif j == 2 and cp == 60:
      break
    elif j == 3 and cp == 26:
      break
    elif j == 4 and cp == 20:
      break
    elif j == 5 and cp == 16:
      break
      
#in ra số phần tử của mỗi list article_link, title, puplishdate
print (len(article_link), len(title))


#**II. Trang báo chính thống**

###*Code dùng chung cho trang báo Chính thống*

- **Cài đặt các gói cần thiết để chạy được trên Colab**

In [ ]:
# install chromium, its driver, and selenium
!apt update
!apt install chromium-chromedriver
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

- **Thư viện chung**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import time

- **Lưu file csv**

In [ ]:
df = pd.DataFrame({'is_sarcastic': 0,
                   'title': title, 
                   'article_link': article_link,
                   'publishdate': publishdate})

df.to_csv("/content/gdrive/MyDrive/Colab Notebooks/Machine Learning/31062021_sarcasm-detection/dataset/<name>.csv", header=True, index=False)
df.to_csv("/content/<name>.csv", header=True, index=False)


##**1. New York Times**

- https://www.nytimes.com/

In [ ]:
# Example url: https://www.nytimes.com/issue/todayspaper/2020/06/03/todays-new-york-times

year = ["2018", "2019", "2020", "2021"]
month = ["12", "11", "10", "09", "08", "07", "06", "05", "04", "03", "02","01"]
date = ["01", "02", "03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28"]

base_url = "https://www.nytimes.com/issue/todayspaper/"


article_link = []
title = []
publishdate = []

for y in year:
    if (y == "2021"):
        month.clear()
        month = ["05", "04", "03", "02","01"]
    for m in month:
        for d in date:
            urls = base_url + y + "/" + m + "/" + d + "/todays-new-york-times"

            browser = webdriver.Chrome('chromedriver',options=options)

            browser.get(urls)

            #scroll page
            scroll_pause_time = 5
            i = 0
            while i < 4:
                if (i != 0):
                    browser.execute_script("window.scrollTo(0, 0);")
                    time.sleep(2)
                    browser.execute_script(
                        "window.scrollTo(0, ((document.body.scrollHeight*9)/10));")
                    # Wait to load page
                    time.sleep(scroll_pause_time)
                else:
                    browser.execute_script(
                        "window.scrollTo(0, (document.body.scrollHeight*3)/4);")
                i += 1

            
            soup = BeautifulSoup(browser.page_source, 'html')
            browser.close()
            detail = soup.findAll('li', attrs={'class':"css-i435f0" })

            for cp in detail:
                title_vs_url = cp.find('div', attrs={'class':"css-141drxa"})
            
                article_link.append("https://www.nytimes.com/" + title_vs_url.a['href'])

                temp_title = title_vs_url.find('h2', attrs={"class":"css-ds6ff4 ehnp9uj0"})
                title.append(temp_title.find(text = True))

                publishdate.append(y + "-" + m + "-" + d)


##**2. The Sunday Times**

- https://www.thetimes.co.uk/

In [ ]:
title = []
article_link = []
publishdate = []

base_url = "https://www.thetimes.co.uk/topic"

#14 topic
topic = [ "uk-politics", "global-politics", 
          "health", "technology", "transport", "science", "law", 
          "economics", "markets", "personal-finance", "real-estate", "banking", 
          "tennis", "golf" ]


for cp in np.arange(1,600):
    for j in range(14):
      url = base_url + "/" + topic[j] + "?page=" + str(cp)

      page = requests.get(url)
      soup = BeautifulSoup(page.text, 'html.parser')

      header = soup.findAll("div", attrs = {"class": "responsiveweb__ListContentContainer-sc-143eup1-0 isZXs responsiveweb__ListContentContainer-sc-143eup1-0 isZXs css-1dbjc4n"})
      text_title = soup.findAll("h3", attrs = {"class": "css-4rbku5 css-901oao r-1khnkhu r-iirzy8 r-evnaw r-16dba41 r-1kt6imw r-d0pm55"})
      date_time = soup.findAll("div", attrs={"class": "css-901oao r-1khp51w r-j2s0nr r-n6v787 r-fxxt2n r-d0pm55"})

      for temp in header:
        if (len(temp.a['href']) > 10):
          article_link.append(temp.a['href'])

      for t in text_title:
        title.append(t.text)

      for x in date_time:
        publishdate.append(x.time['datetime'])
       

print (len(title), len(article_link), len(publishdate))

##**3. The Washington Post**

- https://www.washingtonpost.com/

- **Code Scrapy + Selenium:**

In [ ]:

import scrapy
from selenium import webdriver


class WashingtonPostspider(scrapy.Spider):
    name = 'washingtonpost'
    allowed_domains = ['washingtonpost.com']
    start_urls = ['https://www.washingtonpost.com/politics/?itid=nb_politics']


    def __init__(self):
        self.driver = webdriver.Chrome(
            "C:/Users/Admin/washingtonpost/washingtonpost/chromedriver.exe")

    def parse(self, response):

        self.driver.get(response.url)

        for i in range(600):
            load_more = self.driver.find_element_by_xpath('//*[@id="f0hXwRD2lz28ss"]/div/div[1]/div[2]').click()

        
        titles = response.xpath(
                            "//div[@class = 'story-headline']/h2/a/text()").extract()

        article_links = response.xpath(
                            "//div[@class = 'story-headline']/h2/a/@href").extract()

        row_data = zip(titles,article_links)

        for item in row_data:
            scraped_info = {
                            'title': item[0],
                            'article_link': item[1]
            }
            yield scraped_info

        self.driver.close()


- **Code BeautifulSoup + Selenium:**

In [ ]:
#launch url
url_Politics = "https://www.washingtonpost.com/politics/?itid=nb_front_politics"

browser = webdriver.Chrome('chromedriver',options=options)

article_link = []
title = []

for i in range(10):
  browser.get(url_Politics)
  browser.find_element_by_xpath('//*[@id="f0hXwRD2lz28ss"]/div/div[1]/div[2]').click()

time.sleep(5) 

# Create BeautifulSoup object from page source.
soup = BeautifulSoup(browser.page_source, 'html')

      
# Parse and extract the data that you need.
detail = soup.findAll("div", {'class': 'story-headline'})

for cp in detail:
  temp = cp.find('h2', attrs={'class':""})
 
  if (temp != None):
    article_link.append(temp.a['href'])
    title.append(temp.find(text = True))

print (len(title), len(article_link))

##**4. Newsweek**

- https://www.newsweek.com/

- Nhóm chọn trang này để thay thế cho Trang *The Washington Post* do không crawl được dữ liệu

In [ ]:
#link mẫu:
# https://www.newsweek.com/world?page=9

title = []
article_link = []

category = ['world', "business", "tech-science", "culture", "newsgeek", "sports", "health", "opinion", "education"]

for j in category:
  print (j)
  for cp in np.arange(1, 5):
    url = "https://www.newsweek.com/" + category[j] + "?page=" + str(cp)

    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(url,headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page)

    header = soup.findAll('div', attrs={"class": "inner"})

    for i in header:
      t = i.find('h3')
      article_link.append("https://www.newsweek.com" + t.a['href'])
      title.append(t.text)

    print (cp)


#**III. Code nối các file .csv và chuyển sang json**

- **Nối file csv**

In [ ]:
satiric_1 = pd.read_csv("/content/1_TheBurrardStreetJournal.csv")
satiric_2 = pd.read_csv("/content/1_TheBeaverton.csv")
satiric_3 = pd.read_csv("/content/1_TheDailyEr.csv")
satiric = satiric_1.append(satiric_2)
satiric.append(satiric_3)

official_1 = pd.read_csv("/content/0_TheSunTime.csv")
official_2 = pd.read_csv("/content/0_TheNewYorkTime.csv")
official_3 = pd.read_csv("/content/0_TheNewYorkTime.csv")
official = official_2.append(official_1)
official.append(official_3)

satiric.to_csv("Sarcasm_Detection_v1.csv",index=False)
official.to_csv("Sarcasm_Detection_v2.csv",index=False)

* **Chuyển csv sáng json**

In [ ]:
def csv_to_json(csvFilePath, jsonFilePath):
    jsonArray = []
      
    #read csv file
    with open(csvFilePath, encoding='utf-8') as csvf: 
        #load csv file data using csv library's dictionary reader
        csvReader = csv.DictReader(csvf) 

        #convert each csv row into python dict
        for row in csvReader: 
            #add this python dict to json array
            jsonArray.append(row)
  
    #convert python jsonArray to JSON String and write to file
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
        jsonString = json.dumps(jsonArray, indent=4)
        jsonf.write(jsonString)
          
csvFilePath = r"/content/Sarcasm_Detection_v1.csv"
jsonFilePath = r"/content/Sarcasm_Detection_v1.json"
csv_to_json(csvFilePath, jsonFilePath)

csvFilePath = r"/content/Sarcasm_Detection_v2.csv"
jsonFilePath = r"/content/Sarcasm_Detection_v2.json"
csv_to_json(csvFilePath, jsonFilePath)